In [257]:
import numpy as np
import tensorflow as tf
import time

In [258]:
# gpu_available = tf.test.is_gpu_available()
# is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
# is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
# is_cuda_gpu_min_3

In [259]:
train = np.load('../data/freebase15k_numpy/train.npy')
validation = np.load('../data/freebase15k_numpy/validation.npy')
test = np.load('../data/freebase15k_numpy/test.npy')

entities = np.load('../data/freebase15k_numpy/entities.npy')
relations = np.load('../data/freebase15k_numpy/relations.npy')

train.astype(dtype=np.int8)
validation.astype(dtype=np.int8)
test.astype(dtype=np.int8)
relations.astype(dtype=np.int8)


array([[ 0,  1],
       [ 1,  1],
       [ 2,  1],
       ...,
       [62,  1],
       [63, 32],
       [64, 14]], dtype=int8)

In [260]:
iteration = 500
dim = 100
kns = 20
alpha = 0.000001
beta = 0.0000001

nn1 = np.random.uniform(low=-0.5 / 100, high=0.5 / 100, size=(entities.shape[0], 100))
nn0 = np.random.uniform(low=-0.5 / 100, high=0.5 / 100, size=(entities.shape[0], 100))
nn2 = np.random.uniform(low=-0.5 / 100, high=0.5 / 100, size=(relations.shape[0],100))

In [261]:
def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))

In [262]:
def calculateRank(triple):   
    selectedEntities = entities[:,0]
    if(len((np.where((train[:,1] == triple[1]) & (train[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , train[np.where((train[:,1] ==triple[1]) & (train[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((test[:,1] == triple[1]) & (test[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , test[np.where((test[:,1] ==triple[1]) & (test[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((validation[:,1] == triple[1]) & (validation[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , validation[np.where((validation[:,1] ==triple[1]) & (validation[:,2] ==triple[2]))][0], invert =True ))[0]
    couraptedH = np.empty((selectedEntities.shape[0],3))
    couraptedH[:,0] =selectedEntities
    couraptedH[:,1] =triple[1]
    couraptedH[:,2] =triple[2]
    
    selectedEntities = entities[:,0]
    if(len((np.where((train[:,0] == triple[0]) & (train[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , train[np.where((train[:,0] == triple[0]) & (train[:,2] == triple[2]))][0], invert =True ))[0]
    if(len((np.where((test[:,0] == triple[0]) & (test[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , test[np.where((test[:,0] ==triple[0]) & (test[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((validation[:,0] == triple[0]) & (validation[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , validation[np.where((validation[:,0] ==triple[0]) & (validation[:,2] ==triple[2]))][0], invert =True ))[0]
    couraptedT = np.empty((selectedEntities.shape[0],3))
    couraptedT[:,0] = triple[0]
    couraptedT[:,1] = selectedEntities
    couraptedT[:,2] =triple[2]
    courapted = np.concatenate((couraptedH, couraptedT), axis=0)
    courapted = courapted.astype(np.int16)
    courapted = np.vstack([courapted, triple])
    
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(tf.convert_to_tensor(nn1[courapted[:,1]],dtype=tf.float32),tf.convert_to_tensor(tf.transpose(tf.convert_to_tensor(nn0[courapted[:,0]],dtype=tf.float32) + tf.convert_to_tensor(nn2[triple[2]],dtype=tf.float32) ),dtype=tf.float32),axes=1 )))
    
    courapted =np.concatenate((courapted, p[:, None]), axis=1)
    courapted = courapted[np.argsort(courapted[:, -3])]
    return np.where((courapted[:,0] == triple[0]) & (courapted[:,1] == triple[1]))[0][0]

In [263]:
def mrr(ranks):
    inverse = []
    for rank in ranks:
        inverse.append(float(1)/rank)
    summ = sum(inverse)
    return float(1)/len(inverse) * summ

In [264]:
start = time.time()
# for x in range(0 , iteration):
    # random.shuffle(train)
    # for triple in train:
    #     negative_samples = [(int(target), 0) for target in (np.random.choice(entities[:,0],size = kns, p=entities[:,2]))]
    #     classifiers = [(triple[1], 1)] + negative_samples
    #     # random.shuffle(classifiers)
    #     neu1e = np.zeros(dim)
    #     for target, label in classifiers:
    #         _target_index = target
    #         _head_index = triple[0]
    #         _relation_index = triple[2]
    #         z = np.dot(nn1[_target_index], nn0[_head_index] + nn2[_relation_index])
    #         p = sigmoid(z)
    #         g = alpha * (label - p)
    #         g1 = beta * (label - p)
    #         neu1e += (g * nn1[_target_index] + g1 * nn2[_relation_index])
    #         nn1[_target_index] += g * nn0[_head_index]
    #         nn2[_relation_index] += g * nn0[_head_index]
    # print('total one sample calc time =>  ' , time.time() - start)       
    # if x % 10 == 0 and x > 20:
_mrrList = []
# for triple in validation:
_mrrList.append(calculateRank(validation[0,:]))
print('total one sample calc time =>  ' , time.time() - start) 

total one sample calc time =>   2.745074510574341
